In [21]:
import pandas as pd
import requests
import time

# 填入自己的 Mapbox Access Token
MAPBOX_TOKEN = "(輸入gimini api)"

# 讀取原始資料
df = pd.read_csv("/Users/wanghao/Downloads/zomato_with_delayed_label.csv")


# 準備儲存每筆預估時間（分鐘）
estimated_durations = []

# 記錄執行開始時間
start_time = time.time()

# 遍歷每一筆資料，查詢 Directions API
for idx, row in df.iterrows():
    start_lon = row["Restaurant_longitude"]
    start_lat = row["Restaurant_latitude"]
    end_lon = row["Delivery_location_longitude"]
    end_lat = row["Delivery_location_latitude"]

    url = f"https://api.mapbox.com/directions/v5/mapbox/driving/{start_lon},{start_lat};{end_lon},{end_lat}"
    params = {
        "access_token": MAPBOX_TOKEN,
        "overview": "simplified"
    }

    try:
        res = requests.get(url, params=params)
        data = res.json()

        if "routes" in data and len(data["routes"]) > 0:
            duration = round(data["routes"][0]["duration"] / 60)  # 秒 → 分鐘
        else:
            print(f"❗ 第 {idx} 筆查無路線")
            duration = None

    except Exception as e:
        print(f"⚠️ 錯誤發生在第 {idx} 筆：{e}")
        duration = None

    estimated_durations.append(duration)
    time.sleep(0.2)  # 避免觸發 API 限速

# 加回到原始資料
df["Estimated_duration_minutes"] = estimated_durations

# 儲存為新 CSV 檔案
df.to_csv("zomato_with_estimated_durations.csv", index=False)

# 顯示總耗時
end_time = time.time()
total_sec = int(end_time - start_time)
print(f"✅ 處理完成，共花費 {total_sec // 60} 分 {total_sec % 60} 秒")


✅ 處理完成，共花費 222 分 33 秒
